# Sukurkite regresijos modelį: paruoškite ir vizualizuokite duomenis

## **Linijinė regresija moliūgams - Pamoka 2**
#### Įvadas

Dabar, kai turite įrankius, reikalingus pradėti kurti mašininio mokymosi modelius naudojant Tidymodels ir Tidyverse, esate pasiruošę pradėti užduoti klausimus savo duomenims. Dirbant su duomenimis ir taikant ML sprendimus, labai svarbu mokėti užduoti tinkamus klausimus, kad galėtumėte tinkamai atskleisti savo duomenų potencialą.

Šioje pamokoje išmoksite:

-   Kaip paruošti duomenis modelio kūrimui.

-   Kaip naudoti `ggplot2` duomenų vizualizacijai.

Klausimas, į kurį norite gauti atsakymą, nulems, kokio tipo ML algoritmus naudosite. O atsakymo kokybė labai priklausys nuo jūsų duomenų pobūdžio.

Pažiūrėkime, kaip tai veikia praktiniame uždavinyje.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Piešinys @allison_horst</figcaption>


<!--![Piešinys \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.lt.jpg)<br>Piešinys \@allison_horst-->


## 1. Moliūgų duomenų importavimas ir Tidyverse įtraukimas

Šiai pamokai analizuoti ir apdoroti mums reikės šių paketų:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) yra [R paketų rinkinys](https://www.tidyverse.org/packages), sukurtas tam, kad duomenų mokslas būtų greitesnis, paprastesnis ir smagesnis!

Galite juos įdiegti taip:

`install.packages(c("tidyverse"))`

Žemiau pateiktas scenarijus patikrina, ar turite šiam moduliui reikalingus paketus, ir, jei kai kurių trūksta, juos įdiegia.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Dabar paleiskime keletą paketų ir įkelkime [duomenis](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv), pateiktus šiai pamokai!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Greitas `glimpse()` iš karto parodo, kad yra tuščių reikšmių ir maišytų duomenų tipų: tekstinių (`chr`) ir skaitinių (`dbl`). `Date` yra tekstinio tipo, o taip pat yra keista stulpelis, vadinamas `Package`, kur duomenys yra maišyti tarp `sacks`, `bins` ir kitų reikšmių. Duomenys, tiesą sakant, yra šiek tiek netvarkingi 😤.

Iš tiesų, retai pasitaiko gauti duomenų rinkinį, kuris būtų visiškai paruoštas naudoti kuriant ML modelį iš karto. Bet nesijaudinkite, šiame pamokoje išmoksite, kaip paruošti neapdorotą duomenų rinkinį naudojant standartines R bibliotekas 🧑‍🔧. Taip pat išmoksite įvairių technikų, kaip vizualizuoti duomenis.📈📊
<br>

> Primename: Vamzdžio operatorius (`%>%`) atlieka operacijas logine seka, perduodamas objektą į priekį į funkciją arba iškvietimo išraišką. Galite galvoti apie vamzdžio operatorių kaip "ir tada" savo kode.


## 2. Patikrinkite, ar nėra trūkstamų duomenų

Viena iš dažniausiai pasitaikančių problemų, su kuriomis susiduria duomenų mokslininkai, yra neišsamūs arba trūkstami duomenys. R trūkstamas arba nežinomas reikšmes žymi specialia žymeklio reikšme: `NA` (Not Available).

Taigi, kaip sužinoti, ar duomenų rėmelis turi trūkstamų reikšmių?
<br>
- Vienas paprastas būdas būtų naudoti bazinę R funkciją `anyNA`, kuri grąžina loginį objektą `TRUE` arba `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Puiku, atrodo, kad trūksta kai kurių duomenų! Tai gera vieta pradėti.

-   Kitas būdas būtų naudoti funkciją `is.na()`, kuri nurodo, kurie atskiri stulpelio elementai yra trūkstami, pažymėdama juos logine reikšme `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Gerai, darbas atliktas, tačiau su tokia didele duomenų lentele būtų neefektyvu ir praktiškai neįmanoma peržiūrėti visų eilučių ir stulpelių atskirai😴.

-   Intuityvesnis būdas būtų apskaičiuoti trūkstamų reikšmių sumą kiekvienam stulpeliui:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Daug geriau! Trūksta duomenų, bet galbūt tai nesutrukdys atlikti užduotį. Pažiūrėkime, ką atneš tolimesnė analizė.

> Be nuostabių paketų ir funkcijų rinkinių, R turi labai gerą dokumentaciją. Pavyzdžiui, naudokite `help(colSums)` arba `?colSums`, kad sužinotumėte daugiau apie šią funkciją.


## 3. Dplyr: Duomenų manipuliavimo gramatika

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Piešinys sukurtas @allison_horst</figcaption>


<!--![Piešinys sukurtas \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.lt.png)<br/>Piešinys sukurtas \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), Tidyverse paketas, yra duomenų manipuliavimo gramatika, kuri siūlo nuoseklų veiksmų rinkinį, padedantį spręsti dažniausiai pasitaikančius duomenų manipuliavimo iššūkius. Šioje dalyje apžvelgsime keletą dplyr veiksmų!


#### dplyr::select()

`select()` yra funkcija pakete `dplyr`, kuri padeda pasirinkti stulpelius, kuriuos norite išsaugoti arba pašalinti.

Kad jūsų duomenų rėmelis būtų lengviau naudojamas, pašalinkite kelis jo stulpelius, naudodami `select()`, palikdami tik tuos stulpelius, kurių jums reikia.

Pavyzdžiui, šiame pratime mūsų analizė apims stulpelius `Package`, `Low Price`, `High Price` ir `Date`. Pasirinkime šiuos stulpelius.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` yra funkcija iš paketo `dplyr`, kuri padeda kurti arba keisti stulpelius, išlaikant esamus stulpelius.

Bendras `mutate` struktūros pavyzdys:

`data %>%   mutate(new_column_name = what_it_contains)`

Pažvelkime, kaip veikia `mutate`, naudodami `Date` stulpelį ir atlikdami šiuos veiksmus:

1. Konvertuokime datas (šiuo metu jos yra tipo character) į mėnesio formatą (tai yra JAV datos, todėl formatas yra `MM/DD/YYYY`).

2. Ištraukime mėnesį iš datų į naują stulpelį.

R kalboje paketas [lubridate](https://lubridate.tidyverse.org/) palengvina darbą su datos-laiko duomenimis. Taigi, pasinaudokime `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` ir pažiūrėkime, kaip pasiekti aukščiau nurodytus tikslus. Galime pašalinti `Date` stulpelį, nes jo nebereikės tolimesnėse operacijose.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Oho! 🤩

Dabar sukurkime naują stulpelį `Price`, kuris atspindės vidutinę moliūgo kainą. Dabar paimkime `Low Price` ir `High Price` stulpelių vidurkį, kad užpildytume naująjį stulpelį Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Taip!💪

„Bet palauk!“, pasakysite peržiūrėję visą duomenų rinkinį su `View(pumpkins)`, „Čia kažkas keisto!“🤔

Jei pažvelgsite į `Package` stulpelį, moliūgai parduodami įvairiomis konfigūracijomis. Kai kurie parduodami `1 1/9 bushel` matu, kai kurie `1/2 bushel` matu, kai kurie pagal moliūgą, kai kurie pagal svorį, o kai kurie didelėse dėžėse su skirtingais pločiais.

Patikrinkime tai:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Nuostabu!👏

Moliūgus atrodo labai sunku sverti nuosekliai, todėl filtruokime juos, pasirinkdami tik tuos moliūgus, kurių `Package` stulpelyje yra žodis *bushel*, ir sudėkime tai į naują duomenų rėmelį `new_pumpkins`.


#### dplyr::filter() ir stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): sukuria duomenų poskyrį, kuriame yra tik tos **eilutės**, kurios atitinka jūsų sąlygas, šiuo atveju moliūgai, kurių stulpelyje `Package` yra žodis *bushel*.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): nustato, ar eilutėje yra tam tikras raštas, ar jo nėra.

[`stringr`](https://github.com/tidyverse/stringr) paketas suteikia paprastas funkcijas dažniausiai atliekamoms operacijoms su eilutėmis.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Galite matyti, kad mes susiaurinome iki maždaug 415 eilučių duomenų, kuriuose yra moliūgai pagal statinę.🤩


#### dplyr::case_when()

**Bet palaukite! Yra dar vienas dalykas, kurį reikia padaryti**

Ar pastebėjote, kad kiekis skiriasi kiekvienoje eilutėje? Jums reikia normalizuoti kainodarą, kad būtų rodoma kaina už vieną bušelį, o ne už 1 1/9 ar 1/2 bušelio. Laikas atlikti keletą skaičiavimų, kad tai standartizuotumėte.

Naudosime funkciją [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html), kad *pakeistume* Price stulpelį pagal tam tikras sąlygas. `case_when` leidžia vektorizuoti kelis `if_else()` teiginius.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Dabar galime analizuoti kainas už vienetą, remdamiesi jų matavimu bušeliais. Visa ši bušelių moliūgų analizė, tačiau, parodo, kaip labai `svarbu` yra `suprasti savo duomenų prigimtį`!

> ✅ Pasak [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), bušelio svoris priklauso nuo produkto tipo, nes tai yra tūrio matas. "Pavyzdžiui, pomidorų bušelis turėtų sverti 56 svarus... Lapai ir žalumynai užima daugiau vietos su mažesniu svoriu, todėl špinatų bušelis sveria tik 20 svarų." Viskas gana sudėtinga! Nesivarginkime konvertuoti bušelių į svarus, o vietoj to nustatykime kainą pagal bušelį. Visa ši bušelių moliūgų analizė, tačiau, parodo, kaip labai svarbu yra suprasti savo duomenų prigimtį!
>
> ✅ Ar pastebėjote, kad moliūgai, parduodami pusbušeliais, yra labai brangūs? Ar galite suprasti kodėl? Užuomina: maži moliūgai yra daug brangesni nei dideli, tikriausiai todėl, kad jų bušelyje yra daug daugiau, atsižvelgiant į nepanaudotą vietą, kurią užima vienas didelis tuščiaviduris pyrago moliūgas.


Dabar galiausiai, dėl nuotykių dvasios 💁‍♀️, perkelkime stulpelį „Mėnuo“ į pirmąją poziciją, t. y. „prieš“ stulpelį „Paketas“.

Funkcija `dplyr::relocate()` naudojama stulpelių pozicijoms pakeisti.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Puikus darbas!👌 Dabar turite švarią, tvarkingą duomenų bazę, ant kurios galite kurti savo naują regresijos modelį!
<br>


## 4. Duomenų vizualizacija su ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografikas sukurtas Dasani Madipalli</figcaption>


<!--![Infografikas sukurtas Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.lt.png){width="600"}-->

Yra *išmintingas* posakis, kuris skamba taip:

> "Paprastas grafikas suteikė daugiau informacijos duomenų analitikui nei bet kuris kitas įrankis." --- John Tukey

Duomenų mokslininko vaidmuo yra parodyti duomenų kokybę ir pobūdį, su kuriais jis dirba. Tam jie dažnai kuria įdomias vizualizacijas, arba diagramas, grafikus ir lenteles, atskleidžiančias skirtingus duomenų aspektus. Tokiu būdu jie gali vizualiai parodyti ryšius ir spragas, kurių kitaip būtų sunku pastebėti.

Vizualizacijos taip pat gali padėti nustatyti, kuris mašininio mokymosi metodas yra tinkamiausias duomenims. Pavyzdžiui, sklaidos diagrama, kuri atrodo kaip linija, rodo, kad duomenys yra tinkami linijinės regresijos užduočiai.

R siūlo keletą sistemų grafikai kurti, tačiau [`ggplot2`](https://ggplot2.tidyverse.org/index.html) yra viena elegantiškiausių ir universaliausių. `ggplot2` leidžia kurti grafikus **sujungiant nepriklausomus komponentus**.

Pradėkime nuo paprastos sklaidos diagramos, kurioje bus Price ir Month stulpeliai.

Taigi šiuo atveju pradėsime nuo [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), pateiksime duomenų rinkinį ir estetinį žemėlapį (su [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), o tada pridėsime sluoksnius (pvz., [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) sklaidos diagramoms.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Ar tai naudingas siužetas 🤷? Ar kas nors jame jus nustebina?

Jis nėra ypač naudingas, nes viskas, ką jis daro, tai rodo jūsų duomenis kaip taškų išsidėstymą tam tikrą mėnesį.
<br>


### **Kaip padaryti tai naudingą?**

Norint, kad diagramos rodytų naudingus duomenis, paprastai reikia kažkaip sugrupuoti duomenis. Pavyzdžiui, mūsų atveju, vidutinės moliūgų kainos nustatymas kiekvienam mėnesiui suteiktų daugiau įžvalgų apie pagrindinius duomenų modelius. Tai veda mus prie dar vieno **dplyr** apžvalgos:

#### `dplyr::group_by() %>% summarize()`

Grupinė agregacija R kalboje gali būti lengvai apskaičiuota naudojant

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` pakeičia analizės vienetą nuo viso duomenų rinkinio į atskiras grupes, tokias kaip mėnesiai.

-   `dplyr::summarize()` sukuria naują duomenų rėmelį su viena stulpeliu kiekvienam grupavimo kintamajam ir viena stulpeliu kiekvienai jūsų nurodytai suvestinės statistikai.

Pavyzdžiui, mes galime naudoti `dplyr::group_by() %>% summarize()` tam, kad sugrupuotume moliūgus pagal **Month** stulpelį ir tada rastume **vidutinę kainą** kiekvienam mėnesiui.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Trumpai!✨

Kategoriniai požymiai, tokie kaip mėnesiai, geriausiai pateikiami naudojant stulpelinę diagramą 📊. Už stulpelinių diagramų atsakingi sluoksniai yra `geom_bar()` ir `geom_col()`. Norėdami sužinoti daugiau, peržiūrėkite `?geom_bar`.

Sukurkime vieną!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Tai naudingesnė duomenų vizualizacija! Panašu, kad didžiausia moliūgų kaina yra rugsėjį ir spalį. Ar tai atitinka jūsų lūkesčius? Kodėl taip arba kodėl ne?

Sveikinimai baigus antrąją pamoką 👏! Jūs paruošėte savo duomenis modelio kūrimui, o tada atskleidėte daugiau įžvalgų naudodami vizualizacijas!



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Dėl svarbios informacijos rekomenduojama profesionali žmogaus vertimo paslauga. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius naudojant šį vertimą.
